### Домашняя работа к уроку № 3:
### Системы управления базами данных MongoDB и Кликхаус в Python

1. Установите MongoDB на локальной машине, а также зарегистрируйтесь в онлайн-сервисе. https://www.mongodb.com/ https://www.mongodb.com/products/compass
2. Загрузите данные который вы получили на предыдущем уроке путем скрейпинга сайта с помощью Buautiful Soup в MongoDB и создайте базу данных и коллекции для их хранения.
3. Поэкспериментируйте с различными методами запросов.
4. Зарегистрируйтесь в ClickHouse.
5. Загрузите данные в ClickHouse и создайте таблицу для их хранения.

#### Выполнение задачи №2: Загрузите данные который вы получили на предыдущем уроке путем скрейпинга сайта с помощью Buautiful Soup в MongoDB и создайте базу данных и коллекции для их хранения.

In [2]:
import json
from pymongo import MongoClient

In [3]:
# Чтение данных из JSON файла
with open("books_from_books.toscrape.com.json", "r", encoding='utf-8') as file:
    data = json.load(file)

In [4]:
### Проверка загрузки на примере первой записи
data[0]

{'Название': 'A Light in the Attic',
 'Цена в фунтах стерлингов': 51.77,
 'Количество в наличии': 22,
 'Описание': "\n    It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock

In [5]:
# Соединение с MongoDB
client = MongoClient('localhost', 27017)
db = client['my_books_database']

In [6]:
# Создание коллекции на основании данных из JSON файла
collection_name = "Список книг от 12-03-2024"
for item in data:
    collection = db[collection_name]
    collection.insert_one(item)

client.close()

#### Выполнение задачи №3: Поэкспериментируйте с различными методами запросов.

In [7]:
# Установление связи с MongoDB
client = MongoClient('localhost', 27017)
db = client['my_books_database']
collection = db['Список книг от 12-03-2024']

In [8]:
#### Просмотр количества документов в коллекции
document_count = collection.count_documents({})
print("Количества документов в коллекции:", document_count)

Количества документов в коллекции: 1033


In [25]:
# Запрос количества и названий книг, дороже 50 фунтов
query = {"Цена в фунтах стерлингов": {"$gt": 50.0}}

# Получения документов, соответствующих запросу
documents = collection.find(query)

print("Количество книг дороже 50 фунтов - ", collection.count_documents(query), ":")

# Отображение названий книг
for document in documents:
    print("Название книги:", document['Название'])

Количество книг дороже 50 фунтов -  204 :
Название книги: A Light in the Attic
Название книги: Tipping the Velvet
Название книги: Soumission
Название книги: Sapiens: A Brief History of Humankind
Название книги: The Black Maria
Название книги: Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)
Название книги: Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991
Название книги: Libertarianism for Beginners
Название книги: Birdsong: A Story in Pictures
Название книги: Aladdin and His Wonderful Lamp
Название книги: Behind Closed Doors
Название книги: Slow States of Collapse: Poems
Название книги: We Love You, Charlie Freeman
Название книги: The Secret of Dreadwillow Carse
Название книги: The Pioneer Woman Cooks: Dinnertime: Comfort Classics, Freezer Food, 16-Minute Meals, and Other Delicious Ways to Solve Supper!
Название книги: The Past Never Ends
Название книги: The Murder That Never Was (Forensic Instincts #5)
Название книги: The Electric Pencil: Drawi

In [26]:
### Запрос для определения максимальной и минимальной цены книги в коллекции

# Определение конвеера для агрегации данных
pipeline = [
    {"$group": {"_id": None, "max_price": {"$max": "$Цена в фунтах стерлингов"}, "min_price": {"$min": "$Цена в фунтах стерлингов"}}}
]

# Применение конвеера
result = list(collection.aggregate(pipeline))

# Получение минимального и максимального значений
max_price = result[0]["max_price"]
min_price = result[0]["min_price"]

# Печать результатов
print("Минимальная цена:", min_price)
print("Максимальная цена:", max_price)


Минимальная цена: 10.0
Максимальная цена: 59.99


In [28]:
# Создание конвеера для разбиения всех книг по стоимости на три категории: низкая стоимость (дешевле 15 фунтов), средня стоимость (от 15 до 25 фунтов) и высокая стоимость (выше 25 фунтов). Затем подсчет количества книг в каждой категории агрегацией по полю "Количество в наличии"
pipeline_by_group = [
    {
        "$group": {
            "_id": {
                "$cond": {
                    "if": {"$lte": ["$Цена в фунтах стерлингов", 15]},
                    "then": "Низкая стоимость",
                    "else": {
                        "$cond": {
                            "if": {"$lte": ["$Цена в фунтах стерлингов", 25]},
                            "then": "Средная стоимость",
                            "else": "Высокая стоимость"
                        }
                    }
                }
            },
            "Количество в наличии": {"$sum": "$Количество в наличии"}
        }
    }
]

# Выполнение конвеера
result_by_group = list(collection.aggregate(pipeline_by_group))

# Печать результатов
for category in result_by_group:
    print("Категория по стоимости:", category["_id"])
    print("Количество в наличии:", category["Количество в наличии"])
    print()

# Создание конвеера для определения суммарного количества всех книг по полю "Количество в наличии"
pipeline_total_count = [
    {
        "$group": {
            "_id": None,
            "total_count": {"$sum": "$Количество в наличии"}
        }
    }
]

# Выполнение конвеера
result_total_count = list(collection.aggregate(pipeline_total_count))

# Отображение результата
total_count = result_total_count[0]['total_count'] if result_total_count else 0
print("Общее количество книг в наличии:", total_count) #### совпадает с суммой книг в наличии по категориям стоимости

Категория по стоимости: Низкая стоимость
Количество в наличии: 925

Категория по стоимости: Высокая стоимость
Количество в наличии: 6004

Категория по стоимости: Средная стоимость
Количество в наличии: 1727

Общее количество книг в наличии: 8656
